### PitchTrack for Computer Vision (CS-GY 6643) Final Project Report
Ahhyun Moon am12180@nyu.edu
Chase Huang jh9173@nyu.edu
Yukai Xue yx3254@nyu.edu
Ziyi Wang zw3917@nyu.edu

# 0. Video and Paras
1. Video = "ProjectVideos/1.mp4" [ "ProjectVideos/1_x0.5.mp4" ]
roi = (350, 50, 150, 150)
paras = [1.2,10,200,10,3,8]

2. Video = "ProjectVideos/2.mp4" [ "ProjectVideos/2_x0.5.mp4" ]
roi = (700, 280, 800, 500)
paras1 = [1.5,150,45,33,11,15]
paras2 = [1.,30,200,10,13,15]

3. Video = "ProjectVideos/3_curveball.mp4"
roi = (410, 100, 120, 120)
paras = [1.5,10,200,11,4,10]

4. Video = "ProjectVideos/4_4_seam_fastball.mp4"
roi = (320, 80, 150, 150)
paras = [1.5,10,200,11,4,8]

5. Video = "ProjectVideos/5_change_up.mp4"
roi = (330, 65, 150, 150)
paras = [1.5,10,200,18,2,10]

In [ ]:
roi_1 = (350, 50, 150, 150)  #(x_start, y_start, width, height)
paras_1=[1.2,10,200,10,3,8] # e.g. dp=1, minDist=20, param1 = 200, param2 = 11, minRadius=6,maxRadius=10)
path_1 = "ProjectVideos//1.mp4"

roi_2 = (700, 280, 800, 500)  #(x_start, y_start, width, height)
paras_2_1=[1.5,150,45,33,11,15] # e.g. dp=1, minDist=20, param1 = 200, param2 = 11, minRadius=6,maxRadius=10)
paras_2_2=[1.,30,200,10,13,15]
path_2 = "ProjectVideos\\2_x0.5.mp4"
outpath_2 = "4_4_seam_fastball_result.mp4"

roi_3 = (410, 100, 120, 120)  #(x_start, y_start, width, height)
paras_3=[1.5,10,200,11,4,10]  # e.g. dp=1, minDist=20, param1 = 200, param2 = 11, minRadius=6,maxRadius=10)
path_3 = "ProjectVideos/3.mp4"
outpath_3 = "4_4_seam_fastball_result.mp4"

roi_4 = (320, 80, 150, 150)  #(x_start, y_start, width, height)
paras_4=[1.5,10,200,11,4,8] # e.g. dp=1, minDist=20, param1 = 200, param2 = 11, minRadius=6,maxRadius=10)
path_4 = "ProjectVideos/4.mp4"
outpath_14 = "4_4_seam_fastball_result.mp4"

roi_5 = (330, 65, 150, 150)  #(x_start, y_start, width, height)
paras_5=[1.5,10,200,18,2,10] # e.g. dp=1, minDist=20, param1 = 200, param2 = 11, minRadius=6,maxRadius=10)
path_5 = "ProjectVideos/5.mp4"
outpath_5 = "4_4_seam_fastball_result.mp4"

In [ ]:
rois=[roi_1,roi_2,roi_3,roi_4,roi_5]
parass=[paras_1,paras_2_1,paras_3,paras_4,paras_5]
vids=[path_1,path_2,path_3,path_4,path_5]

# 1. Baseball Detection : Hough Circle Transform

In [ ]:
#Project Version: Frame in, Positions out

#cap = cv2.VideoCapture("baseball pitch5_0.5.mp4")
#fgbg = cv2.createBackgroundSubtractorMOG2()
#ret, frame = cap.read()

import cv2
import numpy as np

def object_detection(frame,roi,paras,sub_on,fgbg=None):
    if sub_on:
        fgmask = fgbg.apply(frame)
        foreground = cv2.bitwise_and(frame, frame, mask=fgmask)
        frame = foreground

    #ROI example:(550, 50, 570, 320)
    x_start, y_start, width, height = roi

    # Crop to the ROI
    roi_frame = frame[y_start:y_start+height, x_start:x_start+width]

    # Convert to grayscale
    gray = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (7, 7), 1)

    # Apply thresholding for pixels close to white
    _, thresh = cv2.threshold(blurred, 130, 255, cv2.THRESH_BINARY)

    # Perform Canny edge detection
    edges = cv2.Canny(thresh, 100, 200)

    # Apply Hough Circle Transform
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=paras[0], minDist=paras[1],
                               param1=paras[2], param2=paras[3], minRadius=paras[4], maxRadius=paras[5])

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for i in range(len(circles)):
            # Adjust coordinates to map back to original frame
            (x,y,r) = circles[i]
            circles[i] = (x + x_start, y + y_start, r)
    # print(circles)

    return(circles)

In [ ]:
def draw_circles(origin,frame,roi,paras):
    x_start, y_start, width, height = roi

    # Crop to the ROI
    roi_frame = frame[y_start:y_start+height, x_start:x_start+width]

    # Convert to grayscale
    gray = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (7, 7), 1)

    # Apply thresholding for pixels close to white
    _, thresh = cv2.threshold(blurred, 130, 255, cv2.THRESH_BINARY)

    # For thicker edges
    # dilate = cv2.dilate(thresh, None, iterations=1)

    # Perform Canny edge detection - Not necessary as it is part of cv2.HoughCircles
    edges = cv2.Canny(thresh, 100, 200)

    # Apply Hough Circle Transform
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, dp=paras[0], minDist=paras[1],
                               param1=paras[2], param2=paras[3], minRadius=paras[4], maxRadius=paras[5])

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            cv2.circle(origin, (x + x_start, y + y_start), r, (0, 0, 255), 3)

    cv2.rectangle(origin, (x_start, y_start), (x_start + width, y_start + height), (255, 0, 0), 3)

    return origin

In [ ]:
def object_detection_save(inpath,outpath,roi,paras,sub_on):
    cap = cv2.VideoCapture(inpath)
    fgbg = cv2.createBackgroundSubtractorMOG2()

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(outpath, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if sub_on:
            fgmask = fgbg.apply(frame)
            foreground = cv2.bitwise_and(frame, frame, mask=fgmask)
            processed_frame = draw_circles(frame,foreground,roi,paras)
        else:
            processed_frame = draw_circles(frame,frame,roi,paras)

        out.write(processed_frame)
    cap.release()
    out.release()

def object_detection_show(inpath,roi,paras,sub_on):
    cap = cv2.VideoCapture(inpath)
    fgbg = cv2.createBackgroundSubtractorMOG2()

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if sub_on:
            fgmask = fgbg.apply(frame)
            foreground = cv2.bitwise_and(frame, frame, mask=fgmask)
            processed_frame = draw_circles(frame,foreground,roi,paras)
        else:
            processed_frame = draw_circles(frame,frame,roi,paras)

        cv2.imshow('Detected Circles', processed_frame)
        cv2.waitKey(10) #10 millisecond buffer
        if 0xFF == ord('q'):
            break


    cap.release()
    cv2.destroyAllWindows()

In [ ]:
## Example Usage
roi = (320, 80, 150, 150)  #(x_start, y_start, width, height)
paras=[1.5,10,200,11,4,8] # e.g. dp=1, minDist=20, param1 = 200, param2 = 11, minRadius=6,maxRadius=10)
path = "4_4_seam_fastball_result.mp4"
outpath = "4_4_seam_fastball_result.mp4"

object_detection_save(path,outpath,roi,paras,True)


# 2.Object Tracking : Kalman Filter

In [ ]:
# !pip install filterpy

In [ ]:
from filterpy.kalman import KalmanFilter
from scipy.optimize import linear_sum_assignment
def initialize_kalman():
    kf = KalmanFilter(dim_x=4, dim_z=2)
    dt = 1.0  # time gap

    #  transition matrix
    kf.F = np.array([[1, 0, dt, 0],
                     [0, 1, 0, dt],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])

    #  measurement matrix
    kf.H = np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0]])

    kf.P *= 1000.  #  initial state covariance
    kf.R = np.eye(2) * 10  # measurement noise
    kf.Q = np.eye(4) * 0.1  # process noise

    # dictionary containing the initialized Kalman filter instance
    return {'kf': kf, 'missed_count': 0, 'history': []}

def compute_iou(boxA, boxB):
    ## Compute the intersection over union of two boxes
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def compute_cost_matrix(detections, trackers,adjusted_boxes_resized):
    ## coumpute cost matrix for assign detection to trackers
    cost_matrix = np.zeros((len(detections), len(trackers)), dtype=np.float32)
    for d, det in enumerate(detections):
        for t, trk in enumerate(trackers):
            if 'last_box' in trk:
                iou = compute_iou(adjusted_boxes_resized[d], trk['last_box'])
                cost_matrix[d, t] = 1 - iou
            else:
                cost_matrix[d, t] = np.linalg.norm(np.array(det[:2]) - np.array(trk['kf'].x[:2].reshape(-1)))
    return cost_matrix

def assign_detections_to_trackers(detections, trackers,adjusted_boxes_resized):
    ## assign detection to trackers using hungarian algorithm
    cost_matrix = compute_cost_matrix(detections, trackers,adjusted_boxes_resized)
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    return row_ind, col_ind, cost_matrix

def update_trackers(trackers, adjusted_boxes_resized, row_ind, col_ind):
    matched_indices = set(row_ind)
    matched_trackers = set(col_ind)
    print(adjusted_boxes_resized)
    detections = [[(box[0] + box[2]) / 2, (box[1] + box[3]) / 2] for box in adjusted_boxes_resized]

    # update the existing trackers
    for d, t in zip(row_ind, col_ind):
        trk = trackers[t]
        det = detections[d]
        trk['kf'].update(np.array([[det[0]], [det[1]]]))  # update karlman filter
        trk['missed_count'] = 0  #  reset missed_count
        # trk['last_box'] = adjusted_boxes_resized[d]

    # Evaluate whether to create a tracker for unassociated detections
    for d in range(len(detections)):
        if d not in matched_indices:
            det = detections[d]
            should_create_new_tracker = True
            if should_create_new_tracker:
                kf = initialize_kalman()
                kf['kf'].update(np.array([[det[0]], [det[1]]]))
                new_tracker = {
                    'kf': kf['kf'],
                    'missed_count': 0,
                    # 'last_box':adjusted_boxes_resized[d],
                    'history': [(int(det[0]), int(det[1]))],
                }
                trackers.append(new_tracker)

    # Increase the missed_count for unassigned trackers
    for t, trk in enumerate(trackers):
        if t not in matched_trackers:
            trk['missed_count'] += 1

In [ ]:
import cv2
import imageio
from PIL import Image
import numpy as np

In [ ]:
def visualize_frame(frame,trackers):
    ## Visualize trace and detection boxes
    for trk in trackers:
        if 'history' in trk:
            for pt in trk['history'][-50:]:  # Use 50 history point to represent the trace
                cv2.circle(frame, pt, 4, (150, 123, 238), -1)
                # cv2.circle(frame, pt, 8, (150, 123, 238), -1)

        # Plot the predict points in red
        pred_x, pred_y = int(trk['kf'].x[0]), int(trk['kf'].x[1])
        cv2.circle(frame, (pred_x, pred_y), 6, (255, 0, 0), -1)
        # cv2.circle(frame, (pred_x, pred_y), 10, (255, 0, 0), -1)

    return frame

In [ ]:
## Params
i=2
cap=cv2.VideoCapture(vids[i])   # video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 20
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_path="Results/5.mp4"
out = cv2.VideoWriter(out_path, fourcc, fps, (frame_width, frame_height))

roi = rois[i] # region of interest for object detection (x_start, y_start, width, height)
paras=parass[i]
fgbg = cv2.createBackgroundSubtractorMOG2()
# print(vids[i])
# print(roi)
# print(paras)

trackers = []
frame_count=0

## Perform object detection and traking in video
while True:
    ret, frame = cap.read()
    frame_count+=1
    if not ret:
        break

    pos = object_detection(frame,roi,paras,True,fgbg)
    if pos is not None:
            # pos_count=pos_count+1
            # print(pos)
            bbox=[]
            for i in pos:
                (x, y, r) = i
                bbox.append([x - r, y - r, x + r, y + r])
            #     cv2.circle(frame, (x, y), r, (0, 0, 255), 3)
            detections = [[i[0],i[1]] for i in pos]

            # print(detections)
            row_ind, col_ind, _ = assign_detections_to_trackers(detections, trackers,detections)
            update_trackers(trackers, bbox,row_ind, col_ind)
    else:
        # if frame_count>150 and frame_count<200:
        if frame_count>30:
            frame = visualize_frame(frame, tracker_last)
        out.write(frame)
        continue
    for trk in trackers:
        trk['kf'].predict()
        pred_x, pred_y = int(trk['kf'].x[0]), int(trk['kf'].x[1])
        trk['history'].append((pred_x, pred_y))  # Update history

    trackers = [trk for trk in trackers if trk['missed_count'] < 10]
    tracker_last=trackers
    vis_frame = visualize_frame(frame, trackers)
    cv2.imwrite("test.jpg",vis_frame)
    out.write(vis_frame)
out.release()

[[494, 193, 508, 207], [494, 178, 508, 192], [481, 185, 499, 203]]
[[424, 175, 434, 185]]
[[424, 175, 434, 185]]
[[427, 161, 439, 173]]
[[433, 150, 443, 160]]
[[431, 150, 441, 160]]
[[433, 150, 443, 160]]
[[437, 142, 445, 150]]
[[437, 142, 445, 150]]
[[441, 136, 449, 144]]
[[442, 128, 454, 140]]
[[442, 128, 454, 140]]
[[442, 128, 454, 140]]
[[445, 123, 457, 135]]
[[445, 123, 457, 135]]
[[447, 120, 459, 132]]
[[451, 123, 461, 133]]
[[449, 123, 459, 133]]
[[449, 123, 459, 133]]
[[451, 123, 463, 135]]
[[452, 125, 460, 133]]
[[454, 127, 464, 137]]
[[454, 132, 464, 142]]
[[453, 132, 465, 144]]
[[453, 132, 465, 144]]
[[454, 138, 466, 150]]
[[454, 138, 466, 150]]
[[454, 147, 466, 159]]
[[454, 156, 466, 168]]
[[454, 156, 466, 168]]
[[454, 156, 464, 166]]
[[456, 170, 464, 178]]
[[456, 170, 464, 178]]
[[454, 183, 464, 193]]
[[454, 196, 464, 206]]
[[454, 196, 464, 206]]
[[454, 196, 464, 206]]
[[453, 182, 465, 194]]
[[455, 183, 465, 193]]
[[455, 183, 465, 193]]
[[455, 183, 465, 193]]
